In [1]:
import pandas as pd
from datetime import *
import sqlite3

In [2]:
df = pd.read_excel('../data.xlsx')
df.tail(3)

,Location,Longitude,Latitude,Arrived,Gallons of Gas in Tank at Arrival,Departed,Gallons of Gas in Tank at Departure
13,14,-108.083496,43.245203,7.18.2019 20:27,10.9,7.19.2019 8:14,10.9
14,15,-108.050537,44.465151,7.19.2019 10:07,8.0,7.19.2019 10:27,8.0
15,16,-110.269775,44.504341,7.19.2019 12:49,4.2,NaN,NaN


In [3]:
from math import radians, cos, sin, asin, sqrt
def distance_miles(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    miles = (6371 * c) / 1.609
    return miles

In [8]:
distances = df[['Longitude', 'Latitude']]
times = df[['Arrived', 'Departed']]
gallons = df[['Gallons of Gas in Tank at Arrival', 'Gallons of Gas in Tank at Departure']]

In [9]:
cont = 1
distances = distances.values
distances_miles = list()

for i in range(len(distances)):
    distances_miles.append(distance_miles(float(distances[i][0]), float(distances[i][1]), float(distances[i+1][0]), float(distances[i+1][1])))
    
    if cont == len(distances) - 1:
        break

    cont += 1

distances_miles, len(distances_miles)


([254.6638030824697,
  69.54610671437179,
  152.0370151227768,
  117.19970901131127,
  98.87533747081315,
  129.05526617856293,
  101.6653414242826,
  119.0641122816983,
  72.3387413758515,
  104.78391009056743,
  125.46465367781154,
  221.2040272111314,
  154.19213896540307,
  84.32427868534609,
  109.44806800428725],
 15)

In [10]:
f = '%m.%d.%Y %H:%M'

times = times.values
cont = 1 
hours = list()
for i in range(len(times)):
    diff = (datetime.strptime(times[i+1][0], f) - datetime.strptime(times[i][1], f)).total_seconds()
    hours.append(diff / (60 * 60))

    if cont == len(times) - 1:
        break
    cont += 1
    
hours, len(hours)

([5.483333333333333,
  1.3666666666666667,
  2.35,
  2.5166666666666666,
  1.6,
  2.7,
  1.5833333333333333,
  2.183333333333333,
  1.1333333333333333,
  1.85,
  2.2666666666666666,
  3.9833333333333334,
  2.4,
  1.8833333333333333,
  2.3666666666666667],
 15)

In [13]:
cont = 1
gallons = gallons.values
spent_gallons = list()

for i in range(len(gallons)):
    spent_gallons.append(gallons[i][1] - gallons[i+1][0])
    
    if cont == len(gallons)-1:
        break

    cont += 1

spent_gallons, len(spent_gallons)


([9.2,
  2.3000000000000007,
  5.1,
  4.5,
  3.0,
  4.800000000000001,
  3.299999999999999,
  3.8000000000000007,
  2.5,
  3.3999999999999995,
  4.4,
  7.1,
  5.1,
  2.9000000000000004,
  3.8],
 15)

In [17]:
conn = sqlite3.connect('../database.db')
cursor = conn.cursor()

In [15]:
consult = '''
insert or ignore into test (id, distance, time, gallons, miles_hour) values (?, ?, ?, ?, ?)
'''

for i in range(len(hours)):
    mh = distances_miles[i] / hours[i]
    cursor.execute(consult, (i, distances_miles[i], round(hours[i], 3), round(spent_gallons[i], 2), mh))
    conn.commit()


In [ ]:
cursor.execute('SELECT distance, time, gallons, MAX(miles_hour) FROM infos')
cursor.fetchall()

In [16]:
cursor.close()
conn.close()